In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils import shuffle


In [3]:
# Path to folder with all CSVs
folder_path = '/Users/user/Downloads/Table/downloaded_csvs_merge'

# Find all CSV files
csv_files = glob(os.path.join(folder_path, '*.csv'))

# Load and concatenate all CSVs
df_list = []
for file in csv_files:
    try:
        df_chunk = pd.read_csv(file)
        df_list.append(df_chunk)
    except Exception as e:
        print(f"Failed to load {file}: {e}")

df = pd.concat(df_list, ignore_index=True)
print("Combined Data Shape:", df.shape)


Combined Data Shape: (45019243, 40)


In [5]:
threshold = 0.9
to_drop = []

for col in df.columns:
    if df[col].isnull().mean() > threshold or (df[col] == 0).mean() > threshold:
        to_drop.append(col)

df = df.drop(columns=to_drop)
print(f"Remaining features: {df.shape[1]}")


Remaining features: 29


In [7]:
df = df.dropna()
df.reset_index(drop=True, inplace=True)
print("Shape after dropping nulls:", df.shape)


Shape after dropping nulls: (45018564, 29)


In [9]:
print(df['Label'].value_counts())


Label
DDOS-ICMP_FLOOD            6893203
DDOS-UDP_FLOOD             5180997
DDOS-TCP_FLOOD             4306058
DDOS-PSHACK_FLOOD          3920337
DDOS-SYN_FLOOD             3886105
DDOS-RSTFINFLOOD           3872787
DDOS-SYNONYMOUSIP_FLOOD    3445630
DOS-UDP_FLOOD              3177280
DOS-TCP_FLOOD              2558232
DOS-SYN_FLOOD              1942159
BENIGN                     1051313
MIRAI-GREETH_FLOOD          949325
MIRAI-UDPPLAIN              852632
MIRAI-GREIP_FLOOD           719604
DDOS-ICMP_FRAGMENTATION     433115
VULNERABILITYSCAN           357579
MITM-ARPSPOOFING            294451
DDOS-UDP_FRAGMENTATION      274881
DDOS-ACK_FRAGMENTATION      272767
DNS_SPOOFING                171463
RECON-HOSTDISCOVERY         128676
RECON-OSSCAN                 93966
RECON-PORTSCAN               78729
DOS-HTTP_FLOOD               68798
DDOS-HTTP_FLOOD              27597
DDOS-SLOWLORIS               22399
DICTIONARYBRUTEFORCE         12522
BROWSERHIJACKING              5630
COMMANDINJECTI

In [7]:
# 🔹 Step 1: Map detailed attack types to broad classes
label_map = {
    'BENIGN': 'BENIGN',
    'DDOS-ICMP_FLOOD': 'DDOS',
    'DDOS-UDP_FLOOD': 'DDOS',
    'DDOS-TCP_FLOOD': 'DDOS',
    'DDOS-PSHACK_FLOOD': 'DDOS',
    'DDOS-SYN_FLOOD': 'DDOS',
    'DDOS-RSTFINFLOOD': 'DDOS',
    'DDOS-SYNONYMOUSIP_FLOOD': 'DDOS',
    'DDOS-ICMP_FRAGMENTATION': 'DDOS',
    'DDOS-UDP_FRAGMENTATION': 'DDOS',
    'DDOS-ACK_FRAGMENTATION': 'DDOS',
    'DDOS-HTTP_FLOOD': 'DDOS',
    'DDOS-SLOWLORIS': 'DDOS',
    'DOS-UDP_FLOOD': 'NON-DDOS',
    'DOS-TCP_FLOOD': 'NON-DDOS',
    'DOS-SYN_FLOOD': 'NON-DDOS',
    'DOS-HTTP_FLOOD': 'NON-DDOS',
    'MIRAI-GREETH_FLOOD': 'NON-DDOS',
    'MIRAI-UDPPLAIN': 'NON-DDOS',
    'MIRAI-GREIP_FLOOD': 'NON-DDOS',
    'VULNERABILITYSCAN': 'NON-DDOS',
    'MITM-ARPSPOOFING': 'NON-DDOS',
    'DNS_SPOOFING': 'NON-DDOS',
    'RECON-HOSTDISCOVERY': 'NON-DDOS',
    'RECON-OSSCAN': 'NON-DDOS',
    'RECON-PORTSCAN': 'NON-DDOS',
    'DICTIONARYBRUTEFORCE': 'NON-DDOS',
    'BROWSERHIJACKING': 'NON-DDOS',
    'COMMANDINJECTION': 'NON-DDOS',
    'SQLINJECTION': 'NON-DDOS',
    'XSS': 'NON-DDOS',
    'BACKDOOR_MALWARE': 'NON-DDOS',
    'RECON-PINGSWEEP': 'NON-DDOS',
    'UPLOADING_ATTACK': 'NON-DDOS'
}

df['BroadLabel'] = df['Label'].map(label_map)

# 🔹 Step 2: Check for any unmapped labels
if df['BroadLabel'].isnull().sum() > 0:
    print("⚠️ Unmapped labels:")
    print(df[df['BroadLabel'].isnull()]['Label'].value_counts())

# 🔹 Step 3: Filter out null mappings
df = df[df['BroadLabel'].notnull()]

# 🔹 Step 4: Sample 1 million from each major class
from sklearn.utils import shuffle

sampled_df = pd.DataFrame()
for broad_class in ['BENIGN', 'DDOS', 'NON-DDOS']:
    subset = df[df['BroadLabel'] == broad_class]
    sample_n = min(1_000_000, len(subset))
    sampled_subset = subset.sample(n=sample_n, random_state=42)
    sampled_df = pd.concat([sampled_df, sampled_subset], axis=0)

sampled_df = shuffle(sampled_df).reset_index(drop=True)
print("✅ Sampled dataset shape:", sampled_df.shape)
print("✅ Distribution:\n", sampled_df['BroadLabel'].value_counts())

⚠️ Unmapped labels:
Series([], Name: count, dtype: int64)
✅ Sampled dataset shape: (3000000, 30)
✅ Distribution:
 BroadLabel
DDOS        1000000
NON-DDOS    1000000
BENIGN      1000000
Name: count, dtype: int64


In [9]:
# 🔹 Step 5: Remove duplicate rows based on feature columns
# Keep only numeric or useful features (exclude Label/BroadLabel)
feature_columns = sampled_df.select_dtypes(include=['number']).columns.tolist()
dedup_df = sampled_df.drop_duplicates(subset=feature_columns)
print("✅ Shape after deduplication:", dedup_df.shape)

✅ Shape after deduplication: (2323194, 30)


In [11]:
# 🔹 Step 6: Drop columns with all 0/null or too many nulls
threshold = 0.98  # if more than 98% are 0 or NaN
to_drop = []

for col in feature_columns:
    if sampled_df[col].isnull().mean() > threshold or (sampled_df[col] == 0).mean() > threshold:
        to_drop.append(col)

print(f"⚠️ Dropping {len(to_drop)} low-variance or null-heavy columns: {to_drop}")
clean_df = dedup_df.drop(columns=to_drop)

⚠️ Dropping 0 low-variance or null-heavy columns: []


In [13]:
# 🔹 Step 7: Log normalization (on numeric columns only)
import numpy as np

numeric_cols = clean_df.select_dtypes(include=['number']).columns
for col in numeric_cols:
    clean_df[col] = clean_df[col].apply(lambda x: np.log1p(x))  # log1p handles 0 safely

print("✅ Log normalization applied.")
print("✅ Final preprocessed dataset shape:", clean_df.shape)

✅ Log normalization applied.
✅ Final preprocessed dataset shape: (2323194, 30)


In [15]:
# 🔹 Save the final cleaned and normalized dataset to CSV
output_path = "0final_cleaned_dataset.csv"
clean_df.to_csv(output_path, index=False)

print(f"✅ Dataset saved to '{output_path}' with shape {clean_df.shape}")


✅ Dataset saved to '0final_cleaned_dataset.csv' with shape (2323194, 30)
